# Chunker.py: Option 2 Implementation Demo

In [2]:
from option2_rnn import *
import os

## Run the default solution on dev

In [3]:
chunker = LSTMTagger(os.path.join('data', 'train.txt.gz'), os.path.join('data', 'chunker'), '.tar')
decoder_output = chunker.decode('data/input/dev.txt')

100%|██████████████████████████████████████████████████████████████████████████████████████████| 1027/1027 [00:02<00:00, 500.06it/s]


## Evaluate the output

In [4]:
flat_output = [ output for sent in decoder_output for output in sent ]
import conlleval
true_seqs = []
with open(os.path.join('data','reference','dev.out')) as r:
    for sent in conlleval.read_file(r):
        true_seqs += sent.split()
conlleval.evaluate(true_seqs, flat_output)

processed 23663 tokens with 11896 phrases; found: 12081 phrases; correct: 9342.
accuracy:  87.17%; (non-O)
accuracy:  88.22%; precision:  77.33%; recall:  78.53%; FB1:  77.92
             ADJP: precision:  51.72%; recall:  19.91%; FB1:  28.75  87
             ADVP: precision:  70.77%; recall:  46.23%; FB1:  55.93  260
            CONJP: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
             INTJ: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
               NP: precision:  76.38%; recall:  82.51%; FB1:  79.33  6737
               PP: precision:  90.87%; recall:  87.26%; FB1:  89.03  2344
              PRT: precision:  71.79%; recall:  62.22%; FB1:  66.67  39
             SBAR: precision:  81.30%; recall:  42.19%; FB1:  55.56  123
               VP: precision:  68.61%; recall:  74.18%; FB1:  71.28  2491


(77.32803575862926, 78.53059852051109, 77.92467781624056)

## Option 1 Implementation (Old)

Firstly, in the process of doing this assignment, we successfully implemented our Option 1 (baseline) implementation which involved implementing the function char_lvl_representation which makes a character level representation of the work. Here v1 is a one-hot vector for the word's first character. v2 contains the count of each character in the word as well as the index of all the characters in between them. The final character of the word is represented by the one-hot vector v3.
makes a character level representation of the work. Here v1 is a one-hot vector for the word's first character. v2 contains the count of each character in the word as well as the index of all the characters in between them. The final character of the word is represented by the one-hot vector v3. The implementation was as follows:

In [5]:
 def char_lvl_representation(sentence):
    input_vectors = []
    size = 100
    for element in sentence:
        v1 = torch.zeros(size)
        v2 = torch.zeros(size)
        v3 = torch.zeros(size)

        if element is not '[UNK]':
            v1[string.printable.find(element[0])] = 1
            v3[string.printable.find(element[-1])] = 1
            for char_unq in (list(set(element[1:-1]))):
                v2[string.printable.find(char_unq)] = element.count(char_unq)
    # append to input vectors
        input_vectors.append(torch.cat((v1, v2, v3), 0))
    return torch.stack(input_vectors)

<>:9: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:9: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_16122/2391541655.py:9: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if element is not '[UNK]':


## Option 2 Implementation (Current)

In order to implement Option 2, building on the existing baseline implementation we created a second RNN that receives the character vector representation as input and produces its hidden state after concatenating it with the word embeddings. This output then goes through the LSTM.
In order to be fed into the RNN cell alongside the initial input in the sequence, a hidden state is seeded as a matrix of zeros in the first step. The input data will be multiplied by the hidden state using weight matrices. In order to induce non-linearity, the product of these multiplications will then be sent through an activation function (here we have tanh function). This reveals the RNN cell's hidden state. 

In [ ]:
class secondRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(secondRNN, self).__init__()
    #  assigning to relavent variables
        self.w_h = nn.Linear(input_dim, hidden_dim)
        self.w_i = nn.Linear(hidden_dim, hidden_dim)
        self.Tan_h = nn.Tanh()
    # forward function
    def forward(self, input_sequence, hidden_p):
        hid = self.Tan_h(self.w_h(input_sequence) + self.w_i(hidden_p))
        return hid

## Analysis

After originally implementing Option 1 (baseline) successfully, our dev.out score increased from 72.70 to 77.21, which is a ~6.20% increase in performance. To further increase the performance of our model, we opted to try to implement the Option 2 improvement which involved developing a second RNN to concatenate the word embeddings and character vectors to add an element of preprocessing of the input to the Chunker RNN. We tested different hidden layer sizes for the RNN, and ultimately we achieved the best dev score of 77.92 with a size of 64, which is a ~7.18% increase in performance over the baseline and a ~0.92% increase in overall performance over the Option 1 implementation.